In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import linregress
import sys
import os

In [7]:
csv_path = "Resources/data.csv"
data_df = pd.read_csv(csv_path)

practice_df = data_df.sample(20)
practice_df

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
79930,0.212,1958,0.975000,"['Ella Fitzgerald', 'Paul Weston And His Orche...",0.287,197347,0.0985,0,4MqEYtEBUH7VnJmhc6xfxX,0.000003,10,0.1060,-16.660,1,How About Me?,20,1958-01-01,0.0321,72.557
139136,0.420,2011,0.020700,"['Justin Bieber', 'Boyz II Men']",0.658,185560,0.6790,0,5kq3YD407av1s3ZHMaZIGv,0.000003,10,0.1980,-8.606,0,Fa La La,49,2011-01-01,0.0361,94.489
65232,0.732,1970,0.990000,['Vashti Bunyan'],0.600,141893,0.1070,0,2vNuzzaFEPY2WGdYFhdeui,0.011400,0,0.1010,-19.189,0,Where I Like To Stand,29,1970,0.0525,117.630
121591,0.312,2004,0.730000,['Landy Wen'],0.692,240653,0.5260,0,7y1At0wJ9E11vQvD93uG9c,0.000000,2,0.0977,-4.065,1,夏日の風,55,2004-07-06,0.0263,113.939
124544,0.249,2019,0.000032,['Coordak'],0.458,215171,0.9330,0,2ytnGi1p2SvgFYIc0H20sV,0.951000,2,0.0810,-5.091,1,Captain,0,2019-01-01,0.0927,144.968
83648,0.930,1976,0.720000,['Los Rancheritos Del Topo Chico'],0.760,176120,0.4690,0,4CdwO4Ga2upRNkTyKN1X6e,0.000001,5,0.0317,-9.866,1,Pa' Que Y Por Qué,35,1976-01-01,0.0395,94.558
61871,0.764,1954,0.299000,['Roy Haynes'],0.608,302787,0.6010,0,2HP7gWK6NAmmxgHfDXwsMi,0.000003,7,0.1090,-10.419,0,Miss Mopsy,9,1954-01-01,0.0347,121.914
152244,0.647,2003,0.724000,['Wendell B'],0.752,302000,0.4950,0,4cNeQgQzEOZv4UEYLIUZUP,0.000000,0,0.3620,-6.743,1,Stl. Thang,37,2003,0.1870,78.010
14580,0.370,1994,0.250000,['Crash Test Dummies'],0.580,235173,0.5750,0,76nvqWPFucUra1xCkN1tnD,0.000018,3,0.0813,-7.720,1,Mmm Mmm Mmm Mmm,55,1994-03-28,0.0231,92.934
121100,0.779,2001,0.413000,['Los Rehenes'],0.640,163200,0.5310,0,4rSYamdhfw8B6VNzQWMHDr,0.000407,11,0.0874,-6.632,1,Para Ti No Tengo Amor,42,2001-01-01,0.0263,86.904


In [72]:
artist_df = pd.DataFrame(practice_df['artists'])
artist_df

for index, row in artist_df.iterrows():
    artist_df["collaborators"] = row['artists'].split(",")

artist_df
# values in column "artists" are strings, not lists. convert to lists and then use len() to count items.


ValueError: Length of values does not match length of index